In [1]:
import numpy as np
import pandas as pd
import pymysql
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

StandardScaler = StandardScaler()

# 엑셀파일을 mysql의 테이블로 구축

In [2]:
data = pd.read_excel('db_score_3_labels.xlsx')
db = pymysql.connect(host='localhost',port=3306,user='root',passwd='iscream1023',db='mysql')
cursor = db.cursor()
query = """CREATE TABLE IF NOT EXISTS db_score2(
            sno int primary key,
            homework float,
            discussion int,
            midterm float,
            grade varchar(10))"""
cursor.execute(query)
db.commit()

In [3]:
query = "INSERT INTO db_score2(sno,homework,discussion,midterm,grade) values "
for i in data.index:
    s = "({0},{1},{2},{3},{4}),".format(data.iloc[i].sno, data.iloc[i].homework, data.iloc[i].discussion, data.iloc[i].midterm, "\'"+data.iloc[i].grade+"\'")
    query += s

cursor.execute(query[:-1])

db.commit()

IntegrityError: (1062, "Duplicate entry '1' for key 'PRIMARY'")

# SVM binary classification

In [2]:
data = pd.read_excel('db_score_3_labels.xlsx')
x=data.drop('grade',axis=1).values
y=np.array([])
for d in data['grade'].values:
    if d == 'B':
        y=np.append(y,1)
    else:
        y=np.append(y,0)

In [5]:
def classification_performance_eval(test,predict):
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for y,p in zip(test,predict):
        if y == 1 and p == 1:
            tp += 1
        elif y == 1 and p == 0:
            fn += 1
        elif y == 0 and p == 1:
            fp += 1
        else:
            tn += 1
    
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    if tp != 0 or fp != 0:
        precision = tp/(tp+fp)
    else:
        precision = 0
    if tp!= 0 or fn != 0:
        recall = tp/(tp+fn)
    else:
        recall = 0
    if precision != 0 or recall != 0:
        f1 = 2*precision*recall/(precision+recall)
    else:
        f1 = 0
        
    return accuracy, precision, recall, f1

In [6]:
import statistics

def print_performance(acc, pre, rec, f1):
    print("average_accuracy : ", statistics.mean(acc))
    print("average_precison : ", statistics.mean(pre))
    print("average_recall : ", statistics.mean(rec))
    print("average_f1_score : ", statistics.mean(f1))

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)
model = SVC(random_state = 42)
model.fit(x_train,y_train)
pred = model.predict(x_test)
accuracy, precision, recall, f1 = classification_performance_eval(y_test,pred)

print("accuracy = %f" %accuracy)
print("precison = %f" %precision)
print("recall = %f" %recall)
print("f1_score = %f" %f1)

accuracy = 0.892857
precison = 0.727273
recall = 1.000000
f1_score = 0.842105


In [10]:
accuracy = []
precision = []
recall = []
f1_score = []

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for t1,t2 in kf.split(x):
    x_train,x_test = x[t1], x[t2]
    y_train,y_test = y[t1], y[t2]
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    acc, pre, rec, f1 = classification_performance_eval(y_test,pred)
    accuracy.append(acc)
    precision.append(pre)
    recall.append(rec)
    f1_score.append(f1)

print_performance(accuracy, precision, recall, f1_score)

average_accuracy :  0.935672514619883
average_precison :  0.8568181818181818
average_recall :  1.0
average_f1_score :  0.918961038961039


# Logistic Regression binary classification

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)

StandardScaler.fit(x_train)
train_data_StandardScale = StandardScaler.transform(x_train)
StandardScaler.fit(x_test)
test_data_StandardScale = StandardScaler.transform(x_test)
model = LogisticRegression(random_state = 42)
model.fit(train_data_StandardScale,y_train)
pred = model.predict(test_data_StandardScale)
accuracy, precision, recall, f1 = classification_performance_eval(y_test,pred)

print("accuracy = %f" %accuracy)
print("precison = %f" %precision)
print("recall = %f" %recall)
print("f1_score = %f" %f1)

accuracy = 0.714286
precison = 0.500000
recall = 0.125000
f1_score = 0.200000


In [12]:
accuracy = []
precision = []
recall = []
f1_score = []

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for t1,t2 in kf.split(x):
    x_train,x_test = x[t1], x[t2]
    y_train,y_test = y[t1], y[t2]
    
    StandardScaler.fit(x_train)
    train_data_StandardScale = StandardScaler.transform(x_train)
    StandardScaler.fit(x_test)
    test_data_StandardScale = StandardScaler.transform(x_test)
    model = LogisticRegression(random_state = 42)
    model.fit(train_data_StandardScale,y_train)
    pred = model.predict(test_data_StandardScale)
    acc, pre, rec, f1 = classification_performance_eval(y_test,pred)
    accuracy.append(acc)
    precision.append(pre)
    recall.append(rec)
    f1_score.append(f1)

print_performance(accuracy, precision, recall, f1_score)

average_accuracy :  0.6181286549707602
average_precison :  0.35
average_recall :  0.12857142857142856
average_f1_score :  0.15714285714285714


# Random Forest binary classification

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)
model = RandomForestClassifier(random_state = 42)
model.fit(x_train,y_train)
pred = model.predict(x_test)
accuracy, precision, recall, f1 = classification_performance_eval(y_test,pred)

print("accuracy = %f" %accuracy)
print("precison = %f" %precision)
print("recall = %f" %recall)
print("f1_score = %f" %f1)

accuracy = 0.964286
precison = 1.000000
recall = 0.875000
f1_score = 0.933333


In [14]:
accuracy = []
precision = []
recall = []
f1_score = []

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for t1,t2 in kf.split(x):
    x_train,x_test = x[t1], x[t2]
    y_train,y_test = y[t1], y[t2]
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    acc, pre, rec, f1 = classification_performance_eval(y_test,pred)
    accuracy.append(acc)
    precision.append(pre)
    recall.append(rec)
    f1_score.append(f1)

print_performance(accuracy, precision, recall, f1_score)

average_accuracy :  0.967251461988304
average_precison :  0.9818181818181818
average_recall :  0.9314285714285714
average_f1_score :  0.9528693528693528


# SVM multi-class classification

In [22]:
def classification_performance_eval2(test,predict):
    arr = []
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for y,p in zip(test,predict):
        if y == 'A' and p == 'A':
            tp += 1
        elif y == 'A' and p != 'A':
            fn += 1
        elif y != 'A' and p == 'A':
            fp += 1
        else:
            tn += 1
    
    
    if tp != 0 or fp != 0:
        precision = tp/(tp+fp)
    else:
        precision = 0
    if tp != 0 or fn != 0:
        recall = tp/(tp+fn)
    else:
        recall = 0
    if precision != 0 or recall != 0:
        f1 = 2*precision*recall/(precision+recall)
    else:
        f1 = 0
    arr.append([precision,recall,f1])
    
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for y,p in zip(test,predict):
        if y == 'B' and p == 'B':
            tp += 1
        elif y == 'B' and p != 'B':
            fn += 1
        elif y != 'B' and p == 'B':
            fp += 1
        else:
            tn += 1
    
    if tp != 0 or fp != 0:
        precision = tp/(tp+fp)
    else:
        precision = 0
    if tp != 0 or fn != 0:
        recall = tp/(tp+fn)
    else:
        recall = 0
    if precision != 0 or recall != 0:
        f1 = 2*precision*recall/(precision+recall)
    else:
        f1 = 0
    arr.append([precision,recall,f1])
    
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for y,p in zip(test,predict):
        if y == 'C' and p == 'C':
            tp += 1
        elif y == 'C' and p != 'C':
            fn += 1
        elif y != 'C' and p == 'C':
            fp += 1
        else:
            tn += 1
    
    if tp != 0 or fp != 0:
        precision = tp/(tp+fp)
    else:
        precision = 0
    if tp != 0 or fn != 0:
        recall = tp/(tp+fn)
    else:
        recall = 0 
    if precision != 0 or recall != 0:
        f1 = 2*precision*recall/(precision+recall)
    else:
        f1 = 0    
    arr.append([precision,recall,f1])
    
    cnt, entire = 0, 0
    for y,p in zip(test,predict):
        entire += 1
        if y == p:
            cnt += 1
    accuracy = cnt/entire
    arr.append([accuracy,0,0])
    
    return arr

def print_multi_performance(arr):
    print("A => precison = %f  recall = %f  f1_score = %f" %(arr[0][0],arr[0][1],arr[0][2]))
    print("B => precison = %f  recall = %f  f1_score = %f" %(arr[1][0],arr[1][1],arr[1][2]))
    print("C => precison = %f  recall = %f  f1_score = %f" %(arr[2][0],arr[2][1],arr[2][2]))
    print("accuracy = %f" %(arr[3][0]))

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)
model = SVC(random_state = 42)
model.fit(x_train,y_train)
pred = model.predict(x_test)
arr = classification_performance_eval2(y_test,pred)

print_multi_performance(arr)

A => precison = 0.909091  recall = 1.000000  f1_score = 0.952381
B => precison = 0.875000  recall = 0.875000  f1_score = 0.875000
C => precison = 1.000000  recall = 0.900000  f1_score = 0.947368
accuracy = 0.928571


In [23]:
performance = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for t1,t2 in kf.split(x):
    x_train,x_test = x[t1], x[t2]
    y_train,y_test = y[t1], y[t2]
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    arr = classification_performance_eval2(y_test,pred)
    arr = np.array(arr)
    performance = performance + arr

performance = performance/5
performance = performance.tolist()
print("performance_mean")
print_multi_performance(performance)

performance_mean
A => precison = 0.742857  recall = 0.771429  f1_score = 0.751282
B => precison = 0.960000  recall = 0.902857  f1_score = 0.926667
C => precison = 0.971429  recall = 1.000000  f1_score = 0.984615
accuracy = 0.956725


# Logistics Regression multi-class classification

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)

StandardScaler.fit(x_train)
train_data_StandardScale = StandardScaler.transform(x_train)
StandardScaler.fit(x_test)
test_data_StandardScale = StandardScaler.transform(x_test)
model = LogisticRegression(random_state = 42)
model.fit(train_data_StandardScale,y_train)
pred = model.predict(test_data_StandardScale)
arr = classification_performance_eval2(y_test,pred)

print_multi_performance(arr)

A => precison = 1.000000  recall = 0.800000  f1_score = 0.888889
B => precison = 0.777778  recall = 0.875000  f1_score = 0.823529
C => precison = 0.909091  recall = 1.000000  f1_score = 0.952381
accuracy = 0.892857


In [25]:
performance = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for t1,t2 in kf.split(x):
    x_train,x_test = x[t1], x[t2]
    y_train,y_test = y[t1], y[t2]
    
    StandardScaler.fit(x_train)
    train_data_StandardScale = StandardScaler.transform(x_train)
    StandardScaler.fit(x_test)
    test_data_StandardScale = StandardScaler.transform(x_test)
    model = LogisticRegression(random_state = 42)
    model.fit(train_data_StandardScale,y_train)
    pred = model.predict(test_data_StandardScale)
    arr = classification_performance_eval2(y_test,pred)
    arr = np.array(arr)
    performance = performance + arr

performance = performance/5
performance = performance.tolist()
print("performance_mean")
print_multi_performance(performance)

performance_mean
A => precison = 0.725000  recall = 0.596429  f1_score = 0.625641
B => precison = 0.620000  recall = 0.640952  f1_score = 0.574545
C => precison = 0.909524  recall = 0.916667  f1_score = 0.907093
accuracy = 0.715205


# Random Forest multi-class classification

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)
model = RandomForestClassifier(random_state = 42)
model.fit(x_train,y_train)
pred = model.predict(x_test)
arr = classification_performance_eval2(y_test,pred)

print_multi_performance(arr)

A => precison = 0.909091  recall = 1.000000  f1_score = 0.952381
B => precison = 1.000000  recall = 0.875000  f1_score = 0.933333
C => precison = 1.000000  recall = 1.000000  f1_score = 1.000000
accuracy = 0.964286


In [27]:
performance = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for t1,t2 in kf.split(x):
    x_train,x_test = x[t1], x[t2]
    y_train,y_test = y[t1], y[t2]
    
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    arr = classification_performance_eval2(y_test,pred)
    arr = np.array(arr)
    performance = performance + arr

performance = performance/5
performance = performance.tolist()
print("performance_mean")
print_multi_performance(performance)

performance_mean
A => precison = 0.766667  recall = 0.800000  f1_score = 0.781818
B => precison = 0.981818  recall = 0.931429  f1_score = 0.952869
C => precison = 0.971429  recall = 0.975000  f1_score = 0.971282
accuracy = 0.967251
